In [1]:
household_id = 'idhogar'
target_column = 'Target'

from data_cleaning import get_training_data, get_test_data, clean_non_numeric_features
train_ind = get_training_data()
test = get_test_data()

train_ind = clean_non_numeric_features(train_ind)

from data_cleaning import target_by_household
target_household_map = target_by_household(train_ind)
train_hh = train_ind.drop(columns=target_column).groupby(household_id).first().astype(int)
train_hh = train_hh.join(target_household_map)

/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Loading data from data/train.csv...
(9557, 142)

Checking for inconsistent targets...
(85,)
(9557, 142)
Cleaning inconsistent targets...
Checking inconsistent targets are gone...
(0,)
(9557, 142)

Loading data from data/test.csv...
(23856, 141)



ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
def get_balanced_validation(df, n, random_state=1):
    return df.sample(frac=1, random_state=random_state).groupby(target_column).head(n)

In [ ]:
valid = get_balanced_validation(train_hh, 100)

In [ ]:
import pandas as pd
X_test = pd.concat([train_hh.reset_index(), valid.reset_index()]).drop_duplicates(keep=False).set_index(household_id)

In [ ]:
import numpy as np
def convert_to_binary_targets(df, true_target):
    df = df.copy()
    df[target_column] = np.where(df[target_column]==true_target, 1, 0)
    return df

In [ ]:
is_4 = convert_to_binary_targets(X_test, 4)

Is it balanced?

In [127]:
from data_cleaning import target_table_breakdown
target_table_breakdown(is_4, {0: 'Not non-vulnerable (target1-3)', 1:'Non vulnerable (target 4)'})

,total,proportion,target description
1,1945,0.659769,Non vulnerable (target 4)
0,1003,0.340231,Not non-vulnerable (target1-3)


Undersample to balance classes

In [128]:
is_4_data = get_balanced_validation(is_4, 1003, random_state=1)

In [129]:
target_table_breakdown(train_hh)

,total,proportion,target description
4,1955,0.654284,non vulnerable households
2,442,0.147925,moderate poverty
3,369,0.123494,vulnerable households
1,222,0.074297,extreme poverty


Feature selection to get best features distinguishing this class

In [130]:
def feature_selector(selector,data, target):
    # Fit the model
    selector.fit(data, target)
    # Retain the feature names
    features = selector.get_support(indices = True)  # Returns array of indexes of nonremoved features
    k_features = [data.columns.values[i] for i in features]
    print(k_features)
    k_features.append(target_column)
    return k_features

In [131]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
sel = SelectKBest(chi2, k=15)
k_features = feature_selector(sel, is_4_data.drop(target_column, axis=1), is_4_data[target_column])
is_4_data = is_4_data[k_features]

['v18q', 'v18q1', 'paredblolad', 'pisocemento', 'cielorazo', 'epared1', 'epared3', 'etecho3', 'eviv1', 'eviv3', 'instlevel1', 'instlevel2', 'instlevel8', 'instlevel9', 'computer']


In [132]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(is_4_data.drop(target_column, axis=1), is_4_data[target_column])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [133]:
is_4_valid = convert_to_binary_targets(valid, 4)
is_4_valid = get_balanced_validation(is_4_valid, 10)
target_table_breakdown(is_4_valid)

,total,proportion,target description
1,10,0.5,extreme poverty
0,10,0.5,None


In [134]:
from sklearn.metrics import classification_report
preds = knn.predict(is_4_valid[k_features].drop(target_column, axis=1))
print(classification_report(is_4_valid[target_column], preds))

             precision    recall  f1-score   support

          0       0.78      0.70      0.74        10
          1       0.73      0.80      0.76        10

avg / total       0.75      0.75      0.75        20



In [135]:
is_3_X_test = X_test.loc[X_test[target_column]!=4]
is_3 = convert_to_binary_targets(is_3_X_test, 3)

In [136]:
sel = SelectKBest(chi2, k=15)
k_features = feature_selector(sel, is_3.drop(target_column, axis=1), is_3[target_column])
is_3 = is_3[k_features]

['v18q', 'v18q1', 'paredblolad', 'paredzinc', 'pisomoscer', 'pisocemento', 'cielorazo', 'energcocinar2', 'energcocinar4', 'epared1', 'epared3', 'eviv1', 'eviv3', 'estadocivil5', 'instlevel6']


In [137]:
target_table_breakdown(is_3, {0: 'In poverty (target1-2)', 1:'Vulnerable (target 3)'})

,total,proportion,target description
0,464,0.633015,In poverty (target1-2)
1,269,0.366985,Vulnerable (target 3)


In [138]:
is_3_data = get_balanced_validation(is_3, 359, random_state=1)

In [139]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(is_3_data.drop(target_column, axis=1), is_3_data[target_column])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [140]:
is_3_valid = valid.loc[valid[target_column]!=4]
is_3_valid = convert_to_binary_targets(is_3_valid, 3)
is_3_valid = get_balanced_validation(is_3_valid, 10)
table = target_table_breakdown(is_3_valid)
print(table['total'].max())
table

10


,total,proportion,target description
1,10,0.5,extreme poverty
0,10,0.5,None


In [141]:
preds = knn.predict(is_3_valid[k_features].drop(target_column, axis=1))
print(classification_report(is_3_valid[target_column], preds))

             precision    recall  f1-score   support

          0       0.44      0.70      0.54        10
          1       0.25      0.10      0.14        10

avg / total       0.34      0.40      0.34        20



In [142]:
def train(clf, train_data, target_value):
    is_n = train_data.loc[train_data[target_column]<=target_value]
    is_n = convert_to_binary_targets(is_n, target_value)
    
    sel = SelectKBest(chi2, k=15)
    k_features = feature_selector(sel, is_n.drop(target_column, axis=1), is_n[target_column])
    is_n = is_n[k_features]
    
    sample_max = target_table_breakdown(is_n)['total'].max()
    is_n = get_balanced_validation(is_n, sample_max, random_state=10)
    clf.fit(is_n.drop(target_column, axis=1), is_n[target_column])
    return clf, k_features

def test(clf, k_features, valid_data, target_value):    
    is_n_valid = valid_data.loc[valid_data[target_column]<=target_value]
    is_n_valid = convert_to_binary_targets(is_n_valid, target_value)
    sample_max = target_table_breakdown(is_n_valid)['total'].max()
    is_n_valid = get_balanced_validation(is_n_valid, sample_max, random_state=10)
    
    preds = clf.predict(is_n_valid[k_features].drop(target_column, axis=1))
    print(classification_report(is_n_valid[target_column], preds))
    
def train_and_test(train_data, valid_data, target_value, clf=KNeighborsClassifier(n_neighbors=20)):
    knn, k_features = train(clf, train_data, target_value)
    test(clf, k_features, valid_data, target_value)

In [143]:
from sklearn.svm import SVC  
svc = SVC(kernel='linear')
train_and_test(X_test, valid, 4, svc)

['v18q', 'v18q1', 'rez_esc', 'paredblolad', 'pisocemento', 'cielorazo', 'epared1', 'epared3', 'etecho1', 'etecho3', 'eviv1', 'instlevel1', 'instlevel2', 'instlevel8', 'computer']
             precision    recall  f1-score   support

          0       0.93      0.40      0.56       300
          1       0.33      0.91      0.49       100

avg / total       0.78      0.53      0.54       400



In [144]:
svc = SVC(kernel='linear')
train_and_test(X_test, valid, 3, svc)

['v18q', 'v18q1', 'paredblolad', 'paredzinc', 'pisomoscer', 'pisocemento', 'cielorazo', 'energcocinar2', 'energcocinar4', 'epared1', 'epared3', 'eviv1', 'eviv3', 'estadocivil5', 'instlevel6']
             precision    recall  f1-score   support

          0       0.67      0.95      0.79       200
          1       0.40      0.06      0.10       100

avg / total       0.58      0.66      0.56       300



In [125]:
svc = SVC(kernel='linear')
train_and_test(X_test, valid, 2, svc)

['r4m1', 'r4t1', 'techocane', 'abastaguafuera', 'elimbasu2', 'epared3', 'etecho3', 'estadocivil1', 'estadocivil5', 'estadocivil6', 'hogar_mayor', 'instlevel1', 'instlevel7', 'computer', 'lugar3']
             precision    recall  f1-score   support

          0       0.40      0.02      0.04       100
          1       0.50      0.97      0.66       100

avg / total       0.45      0.49      0.35       200

